In [29]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [30]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [31]:
torch.manual_seed(42)

In [32]:
data = pd.read_csv("fashion-mnist_train.csv")

In [33]:
data.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
X=data.iloc[:,1:]
Y=data.iloc[:,0]

In [35]:
X=X/255.0

In [36]:
X

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.019608,0.0,0.0,...,0.000000,0.000000,0.000000,0.117647,0.168627,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.003922,0.007843,0.0,0.0,0.000000,0.0,0.0,...,0.011765,0.000000,0.000000,0.000000,0.000000,0.003922,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
59996,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.286275,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
59997,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.627451,0.635294,0.639216,0.529412,0.368627,0.000000,0.0,0.0,0.0,0.0
59998,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


In [37]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [38]:
X_train_tensor= torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor= torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor= torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor= torch.tensor(y_test.values, dtype=torch.long)

In [39]:
class CoustemDataset(Dataset):
  def __init__(self, features, labels):
    self.features=features
    self.labels=labels
  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self, index):
     return self.features[index], self.labels[index]

In [40]:
train_dataset= CoustemDataset(X_train_tensor,y_train_tensor)
test_dataset= CoustemDataset(X_test_tensor,y_test_tensor)

In [41]:
from numpy.random import triangular
train_data_loader= DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_data_loader= DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)

In [42]:
class MYNN(nn.Module):
  def __init__(self,num_features) -> None:
    super().__init__() #invoking the parent class's constructor
    self.model=nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128,64),
        nn.ReLU(),
        nn.Linear(64,10) #we don't need to pass it via softmax because pytorch's cross entropy loss does that automatically
    )

  def forward(self,X):
    return self.model(X)

In [43]:
learning_rate= 0.001
epochs=100

In [44]:
from torch.optim import optimizer
criterion= nn.CrossEntropyLoss()



In [45]:
model= MYNN(X_train_tensor.shape[1])
model=model.to(device)
optimizer= optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
  total_loss=0
  for batch_features, batch_labels in train_data_loader:
    batch_features, batch_labels= batch_features.to(device), batch_labels.to(device)
    optimizer.zero_grad()
    outputs= model(batch_features)
    loss= criterion(outputs, batch_labels)
    loss.backward()
    optimizer.step()
    total_loss+=loss.item()
  avg_loss= total_loss/len(train_data_loader)

  print(f"Epoch: {epoch+1}, Loss: {avg_loss}")

# training loop

# for epoch in range(epochs):

#   total_epoch_loss = 0

#   for batch_features, batch_labels in train_loader:

#     # forward pass
#     outputs = model(batch_features)

#     # calculate loss
#     loss = criterion(outputs, batch_labels)

#     # back pass
#     optimizer.zero_grad()
#     loss.backward()

#     # update grads
#     optimizer.step()

#     total_epoch_loss = total_epoch_loss + loss.item()

#   avg_loss = total_epoch_loss/len(train_loader)
#   print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')

Epoch: 1, Loss: 0.5527251036564509
Epoch: 2, Loss: 0.3926751301934322
Epoch: 3, Loss: 0.3544542966733376
Epoch: 4, Loss: 0.32694049732883773
Epoch: 5, Loss: 0.3071295080582301
Epoch: 6, Loss: 0.29175592111671966
Epoch: 7, Loss: 0.27882406501223644
Epoch: 8, Loss: 0.26615497599542143
Epoch: 9, Loss: 0.2532227461313208
Epoch: 10, Loss: 0.2466269981972873
Epoch: 11, Loss: 0.23765523035948474
Epoch: 12, Loss: 0.22805158472309509
Epoch: 13, Loss: 0.21954751245925824
Epoch: 14, Loss: 0.2133918329825004
Epoch: 15, Loss: 0.20595886832848193
Epoch: 16, Loss: 0.19984006962366402
Epoch: 17, Loss: 0.19495809140677253
Epoch: 18, Loss: 0.18768631747364997
Epoch: 19, Loss: 0.18192669223745664
Epoch: 20, Loss: 0.17795938213790458
Epoch: 21, Loss: 0.17214797662508985
Epoch: 22, Loss: 0.16649255629132192
Epoch: 23, Loss: 0.16363671649247408
Epoch: 24, Loss: 0.16012822507182137
Epoch: 25, Loss: 0.15279452226279924
Epoch: 26, Loss: 0.1520285382559523
Epoch: 27, Loss: 0.1440434282881518
Epoch: 28, Loss: 0.

In [46]:
model.eval()

MYNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [47]:
# evaluation code
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_data_loader:
    batch_features, batch_labels= batch_features.to(device), batch_labels.to(device)
    outputs = model(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)


0.8844166666666666


In [48]:
len(test_data_loader)

375

In [49]:
len(train_data_loader)

1500